In [1]:
###############
### IMPORTS ###
###############

import matplotlib
import pylab

import numpy as np

import scipy as sc
import scipy.stats as stats

import astropy
from astropy import units
from astropy import units as u

import galpy
import galpy.df
from galpy.orbit import Orbit
from galpy.potential import MWPotential2014
from galpy.potential import SpiralArmsPotential, GaussianAmplitudeWrapperPotential, evaluatePotentials, evaluateDensities
from galpy.actionAngle import actionAngleAdiabatic, actionAngleStaeckel
from galpy.util import conversion
from galpy.util.plot import dens2d
from galpy.potential import plotPotentials
from galpy.potential import evaluatePotentials
from galpy.potential import plotRotcurve
from galpy.potential import vcirc
from galpy.potential import lindbladR

from matplotlib.pyplot import *
from matplotlib import pyplot as pl
pl.rc('text', usetex=False)
pl.rc('font', **{'family':'Computer Modern','size':20})
pl.rc('axes', labelsize=16)
pl.rc('xtick',labelsize=16)
pl.rc('ytick',labelsize=16)
from matplotlib import colors as mc
from matplotlib import cm
from matplotlib.colors import LogNorm
from matplotlib.image import NonUniformImage
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

#Run in Physics1 Environment

In [2]:
###############################
### IMPORTANT SCALE FACTORS ###
###############################

# Scales to be use for conversion from natural coordinates
ro = 8.             # [kpc] scale radii
vo = 220.           # [km/s] scale velocity at ro

# Set model parameters for potentials and DF:
hro  = 1./3.        # Scale length for surface density
sro  = 0.16         # Radial velocity dispersion at ro in natural coordinates (times vo to get in physical at ro)
szo  = sro/2        # Vertical velocity dispersion at ro in natural coordinates (times vo to get in physical at ro)
hsro = 3*hro        # Radial scale length for radial velocity dispersion profile
hszo = hsro         # Radial scale length for vertical velocity dispersion profile
sigmaAge = 'sAvg'   # (s2Gyr 0.09) (sAvg sro=0.16) # Set age for velocity dispersion

# Set parameters for initial conditions
nsample=10000         # number of samples
nbatch=10           # Total number of batches assuming start from 1
rmin=0.375          # sets scaled rmin = 3 kpc
rmax=1.875          # sets scaled rmax = 15 kpc
zmax=0.3            # sets scales zmax = 2.4 kpc
AAType = 'stklAA'   # Action Angle approx for ICs

############################
### DISK POTENTIAL SETUP ###
############################

### Set up underlying initial axi-symmetric potential for the disk

dpType = 'MW14'
mwp = MWPotential2014

###############################
### SPIRAL MODEL PARAMETERS ###
###############################

### SET THESE important parameters for evaluation of model

Reval = 1.          # radius at which to evaluate potential (put in natural units)
pdeg = 30.          # pitch angle of the logarithmic spiral arms in degrees
CRs = 1.            # corotation radius (put in natural units)
efrac = 0.3
epsilonSigma = evaluateDensities(mwp,CRs,0.)*efrac  # fractional amplitude of the surface density for spiral strength

##############################
### SPIRAL POTENTIAL SETUP ###
##############################

### Set up 3D transient spiral potential

def degtorad(tdeg): # Convert from degrees to radians
    return tdeg *(2.*np.pi/360.) 
 
# Setup spiral potential from Cox & Gomez (2001)
As = epsilonSigma           # amplitude of potential
Narms = 4                   # number of spiral arms
alpharad = degtorad(pdeg)   # pitch angle of the logarithmic spiral arms in radians 
rref = CRs                  # fiducial radius where rho=rhoo (ro)
phiref = 0.                 # reference angle phi_p(ro)
Rscale = 0.3                # radial scale length of the drop-off in density amplitude of the arms
zscale = 0.0375             # scale height of the stellar arm perturbation (can be Quantity)
v_c = vcirc(mwp,R=CRs)      # v_circ at corotation radius (R in [natunits])
omegas = v_c/CRs            # rotational pattern speed of the spiral arms (can be Quantity) #1./CRs (original)
Cslist = [1]                # list of constants multiplying the cos(n gamma) terms

sp = SpiralArmsPotential(amp=As,N=Narms,alpha=alpharad,
                         r_ref=rref,phi_ref=phiref,
                         Rs=Rscale,H=zscale,omega=omegas,
                         Cs=Cslist)

# Initialized transient spiral as Gaussian growth and decay 
spiT = 2.*np.pi/omegas # Period of rotation for the spiral
tinat = 0.             # Start time
As = 1.                # amplitude to be applied to the potential 

Npeak = 2 # spiT to evolve and then peak spiral
Nsigma = 1 # standard deviation of the Gaussian (one spiral period)
Nequil = 2 # spiT after peak to stop evolution to equilibrate

sigmat = Nsigma*spiT       # standard deviation of the Gaussian (one spiral period) 
to= Npeak*spiT             # Gaussian amplitude peaks at to (peaks at Npeak spiral periods after t=0)

stp = GaussianAmplitudeWrapperPotential(amp=As, pot=sp, to=to, sigma=sigmat)
#have this definition copied to code box right before orbit integration for loop also, or will throw an error

simname = 'R'+str(int(CRs*ro))+'_e'+str(efrac)+'_m'+str(Narms)+'_t'+str(int(pdeg))

In [3]:
nitr=4

for nbatch in range(nitr):

    # Import data
    print('\n Importing Data Files for iteration: ,', nbatch)
    filepath = "C://Users/Flash/OneDrive - brynmawr.edu/AstroLab/Code/galpy_orbits/orbits/"
    actionfile = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_actions_'+str(simname)+'_'+str(nbatch)+'.npy')
    cartfile = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_orbits_cart_'+str(simname)+'_'+str(nbatch)+'.npy')
    cylfile = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_orbits_cyl_'+str(simname)+'_'+str(nbatch)+'.npy')

print('\n Done importing data files')

# for nbatch in range(nitr):
    
#     #create iteration file variable
#     print('Creating File Variables for iteration: ,', nbatch)
#     actionfile+'_'+str(nbatch) = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_actions_'+str(simname)+'_'+str(nbatch)+'.npy')
#     cartfile+'_'+str(nbatch) = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_orbits_cart_'+str(simname)+'_'+str(nbatch)+'.npy')
#     cylfile+'_'+str(nbatch) = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_orbits_cyl_'+str(simname)+'_'+str(nbatch)+'.npy')

# print('\n Done creating file variables')


#Define iteration files

actionfile_0 = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_actions_'+str(simname)+'_'+str(0)+'.npy')
cartfile_0 = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_orbits_cart_'+str(simname)+'_'+str(0)+'.npy')
cylfile_0 = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_orbits_cyl_'+str(simname)+'_'+str(0)+'.npy')
actionfile_1 = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_actions_'+str(simname)+'_'+str(1)+'.npy')
cartfile_1 = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_orbits_cart_'+str(simname)+'_'+str(1)+'.npy')
cylfile_1 = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_orbits_cyl_'+str(simname)+'_'+str(1)+'.npy')
actionfile_2 = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_actions_'+str(simname)+'_'+str(2)+'.npy')
cartfile_2 = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_orbits_cart_'+str(simname)+'_'+str(2)+'.npy')
cylfile_2 = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_orbits_cyl_'+str(simname)+'_'+str(2)+'.npy')
actionfile_3 = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_actions_'+str(simname)+'_'+str(3)+'.npy')
cartfile_3 = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_orbits_cart_'+str(simname)+'_'+str(3)+'.npy')
cylfile_3 = np.load(filepath+str(simname)+'/'+str(N)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(sigmaAge)+'_orbits_cyl_'+str(simname)+'_'+str(3)+'.npy')

    
#Define Actions
jR_f=actionfile_3[0,:]     #final actions after all iterations
jphi_f=actionfile_3[1,:]   #final actions after all iterations
jz_f=actionfile_3[2,:]     #final actions after all iterations
jR_o=actionfile_0[3,:]     #initial actions before any iterations
jphi_o=actionfile_0[4,:]   #initial actions before any iterations
jz_o=actionfile_0[5,:]     #initial actions before any iterations
jR_fg=actionfile_2[3,:]    #actions after growth has stopped
jphi_fg=actionfile_2[4,:]  #actions after growth has stopped
jz_fg=actionfile_2[5,:]    #actions after growth has stopped


 Importing Data Files for iteration: , 0


NameError: name 'N' is not defined

In [ ]:
###################################    
# Plot action final vs initial
###################################

fig1 = pl.figure()
pl.hist2d(jR_o, jR_f, bins=(100,100), norm=LogNorm(), cmap='plasma')
pl.title('JR Final vs JR Initial')
pl.xlabel('jR_o')
pl.ylabel('jR_f')
cbar = pl.colorbar()
cbar.ax.set_ylabel('Counts')
pl.xlim(0,0.3)
pl.ylim(0,0.3)
pl.show()

fig2 = pl.figure()
pl.hist2d(jphi_o, jphi_f, bins=(100,100), norm=LogNorm(), cmap='plasma')
pl.title('Jphi Final vs Jphi Initial')
pl.xlabel('jphi_o')
pl.ylabel('jphi_f')
cbar = pl.colorbar()
cbar.ax.set_ylabel('Counts')
pl.show()

fig3 = pl.figure()
pl.hist2d(jz_o, jz_f, bins=(100,100), norm=LogNorm(), cmap='plasma')
pl.title('Jz Final vs Jz Initial')
pl.xlabel('jz_o')
pl.ylabel('jz_f')
cbar = pl.colorbar()
cbar.ax.set_ylabel('Counts')
pl.xlim(0,0.05)
pl.ylim(0,0.05)
pl.show()

fig4 = pl.figure()
pl.hist2d(jphi_f*ro*vo, jR_f*ro*vo, bins=(100,100), norm=LogNorm(), cmap='plasma')
pl.title('JR Final vs Jphi Final (Phys Units)')
pl.xlabel('jphi_f [kpc km/s]')
pl.ylabel('jR_f [kpc km/s]')
cbar = pl.colorbar()
cbar.ax.set_ylabel('Counts')
# pl.xlim(0,2.0)
pl.ylim(0,400)
pl.show()

fig5 = pl.figure()
pl.hist2d(jphi_f*ro*vo, np.sqrt(jR_f*ro*vo), bins=(100,100), norm=LogNorm(), cmap='plasma')
pl.title('Final $\sqrt{J_{R_{f}}}$ vs Final $J_{\phi_{f}}$')
pl.xlabel('$J_{\phi_{f}}$ (kpc km/s)')
pl.ylabel(r'$\sqrt{J_{R_{f}}}$ $(kpc km/s)^{\frac{1}{2}}$')
cbar = pl.colorbar()
cbar.ax.set_ylabel('Counts')
# pl.xlim(0,2.0)
pl.ylim(0,20)
pl.show()

In [ ]:
#########################################################
###############      Create Function       ##############
###############  to Obtain Radial Action   ##############
###############     from Jacobi Energy     ##############
###############   at Primary Resonances    ##############
#########################################################


from galpy.potential import epifreq


def getJRfromJacobienergy(m, omegas): 
    pot = mwp

#     m = 4
#     ro = 8
#     vo = 220

    OLR = lindbladR(Pot=pot, OmegaP=omegas, m=-m, ro=ro,vo=vo) # m being negative makes this an OLR; in physical units
    ILR = lindbladR(Pot=pot, OmegaP=omegas, m=m, ro=ro,vo=vo) #lindbladR must have ro and vo in order to solve tuple problem
    CR = lindbladR(Pot=pot, OmegaP=omegas, m='corotation', ro=ro,vo=vo)

    pot_ILR = evaluatePotentials(mwp, ILR, z=0, phi=0, ro=ro,vo=vo)
    pot_OLR = evaluatePotentials(mwp, OLR, z=0, phi=0, ro=ro,vo=vo)
    pot_CR = evaluatePotentials(mwp, CR, z=0, phi=0, ro=ro,vo=vo)

#     print ('This is evaluatepotential for ILR:',pot_ILR)
#     print ('\nThis is evaluatepotential for OLR:',pot_OLR)
#     print ('\nThis is evaluatepotential for CR:',pot_CR)

    # print ('This is mwp:',mwp)
    kappa_ILR = epifreq(pot,R=ILR/ro,ro=ro,vo=vo)
#     print ('\nThis is kappa_ILR:',kappa_ILR)

    kappa_OLR = epifreq(pot,R=OLR/ro,ro=ro,vo=vo)
#     print('\nThis is kappa_OLR:',kappa_OLR)

    kappa_CR = epifreq(pot,R=CR/ro,ro=ro,vo=vo)
#     print('\nThis is kappa_CR:',kappa_CR)

    OmegaP = omegas*(vo/ro) #pattern speed at corotation
#     print ('\nThis is OmegaP:',OmegaP)

    v_c_ILR = vcirc(mwp, R = ILR/ro, ro = ro, vo = vo) #in physical units
    OmegaP_ILR = v_c_ILR/ILR

    v_c_OLR = vcirc(mwp, R = OLR/ro, ro = ro, vo = vo) #in physical units
    OmegaP_OLR = v_c_OLR/OLR

    E_c_ILR = pot_ILR + (1/2)*vcirc(mwp,R=ILR/ro,ro=ro,vo=vo)**2
    E_c_OLR = pot_OLR + (1/2)*vcirc(mwp,R=OLR/ro,ro=ro,vo=vo)**2
    E_c_CR = pot_CR + (1/2)*vcirc(mwp,R=CR/ro,ro=ro,vo=vo)**2

#     print('\nThis is KE:', vcirc(mwp,R=ILR/ro,ro=ro,vo=vo)**2)
#     print ('\nThis is circular energy for ILR',E_c_ILR)
#     print ('\nThis is circular energy for OLR',E_c_OLR)
#     print ('\nThis is circular energy for CR',E_c_CR)

    # E_J_ILR = E_c_ILR - OmegaP*ILR*vcirc(mwp,R=ILR/ro,ro=ro,vo=vo)
    # print ('\nThis is Jacobi Integral of ILR', E_J_ILR)

    # E_J_OLR = E_c_OLR - OmegaP*OLR*vcirc(mwp,R=OLR/ro,ro=ro,vo=vo)
    # print ('\nThis is Jacobi Integral of OLR', E_J_OLR)

    # E_J_CR = E_c_CR - OmegaP*CR*vcirc(mwp,R=CR/ro,ro=ro,vo=vo)
    # print ('\nThis is Jacobi Integral of CR', E_J_CR)

    ##### Find y-intercept: J_R = ((E_J(J_R=0) - E_c)/kappa) + (OmegaP/kappa)*Lz #####
    ##### This formula has the form of y = b + m*x 
    ##### (E_J(J_R=0) - E_c)/kappa = (pot +(1/2)*vcirc**2 - OmegaP*Lz - pot - (1/2)*vcirc**2)/kappa 
    #####                          = -OmegaP*Lz/kappa 
    #####                          = -OmegaP*R_g*vcirc/kappa

    y_intercept_ILR = OmegaP*ILR*vcirc(mwp,R=ILR/ro,ro=ro,vo=vo)/kappa_ILR
    y_intercept_OLR = OmegaP*OLR*vcirc(mwp,R=OLR/ro,ro=ro,vo=vo)/kappa_OLR
    y_intercept_CR = OmegaP*CR*vcirc(mwp,R=CR/ro,ro=ro,vo=vo)/kappa_CR

#     print ('\nThis is the y intercept when R = ILR', y_intercept_ILR) 
#     print ('\nThis is the y intercept when R = OLR', y_intercept_OLR) 
#     print ('\nThis is the y intercept when R = CR', y_intercept_CR) 
    ####################################################################################

    Lz = np.linspace(0,2600)

    J_R_ILR = -y_intercept_ILR + (OmegaP*Lz)/kappa_ILR
    J_R_OLR = -y_intercept_OLR + (OmegaP*Lz)/kappa_OLR
    J_R_CR = -y_intercept_CR + (OmegaP*Lz)/kappa_CR

    # plt.plot(Lz,J_R_ILR)
    # plt.plot(Lz,J_R_OLR)
    # plt.plot(Lz,J_R_CR)
    # plt.xlabel('$L_{z}$')
    # plt.ylabel('$J_{R}$')
    
    return J_R_ILR, J_R_OLR, J_R_CR

#getJRfromJacobienergy(4)[0] # This outputs J_R_ILR
# getJRfromJacobienergy(4)[1] # This outputs J_R_OLR
# getJRfromJacobienergy(4)[2] # This outputs J_R_CR

In [ ]:
#############################################################
###############         Create Full            ##############
###############     Orbital Action Graphs      ##############
###############   for All Primary Resonances   ##############
#############################################################


def getangularactiongraph(nbins, colormap, normalize, CRs, m, jphi, jphi_quiet, jr, jr_quiet, cbarlabel_deltajphi, cbarlabel_deltajr, xaxislabel, yaxislabel,vlinelabelILR,vlinelabelOLR):     
           
    ################################
    #########  Parameters  #########
    ################################
    
    normalize = normalize
    colormap = colormap
    
    mwp = MWPotential2014
    ro = 8 #units: kpc
    vo = 220 #units: km/s
    v_c = vcirc(mwp, R = CRs) # v_c in natural units and it's the v_c at corotation radius
    omegas = v_c/CRs # Pattern speed based on CR #orbital freq = v_c/R : we need to figure out v_c at CR
    
    pot = mwp

    OLR = lindbladR(Pot=pot, OmegaP=omegas, m=-m, ro=ro,vo=vo) # m being negative makes this an OLR; in physical units
    ILR = lindbladR(Pot=pot, OmegaP=omegas, m=m, ro=ro,vo=vo) #lindbladR must have ro and vo in order to solve tuple problem
    CR = lindbladR(Pot=pot, OmegaP=omegas, m='corotation', ro=ro,vo=vo) #m=2 is LR and m=4 is Ultraharmonic LR

    Lz = np.linspace(0,2600)
    
    nbins = 201
    
    delta_jphi = jphi - jphi_quiet 
    delta_jr = jr - jr_quiet

    jphimax = jphi_quiet.max()
    jphimin = jphi_quiet.min()
    jrmin = jr_quiet.min()
    jrmax = jr_quiet.max()
    delta_jphimin = delta_jphi.min()
    delta_jphimax = delta_jphi.max()
    
    ####### This block is to make any zeros in the graphs white or blend in with background ########

    for i in range(len(jphi_f)): #was jphii0
        if (jphi_quiet[i]<jphimin or jphi_quiet[i]>jphimax or jr_quiet[i]<jrmin or jr_quiet[i]>jrmax or delta_jphi[i]<delta_jphimin or delta_jphi[i]>delta_jphimax):
            jphi[i]=float("NaN")
        is_nan = (jphi[:] != jphi[:])
        jphi = jphi[~is_nan]
        jr = jr[~is_nan]
        delta_jphi = delta_jphi[~is_nan]
        
    ####### Histogram Setups ########    

    statistic, x_edges, y_edges, binnumber = stats.binned_statistic_2d(jphi_quiet*ro*vo, jr_quiet*ro*vo, delta_jphi*ro*vo, statistic='mean', bins=nbins)
    statistic1, x_edges, y_edges, binnumber = stats.binned_statistic_2d(jphi_quiet*ro*vo, jr_quiet*ro*vo, delta_jr*ro*vo, statistic='mean', bins=nbins)

    x_width = (x_edges[1]-x_edges[0])
    x_centers = x_edges[1:]-x_width/2
    xi = x_centers

    y_width = (y_edges[1]-y_edges[0])
    y_centers = y_edges[1:]-y_width/2
    yi = y_centers

    ji = np.transpose(statistic)
    ji1 = np.transpose(statistic1)

 

    #####################################################
    ##### Plot Jr vs Jphi color coded by delta Jphi #####
    #####################################################
        
        
    fig, ax = pl.subplots()
    # ax.set_aspect(1.)
    change_j = ax.pcolormesh(xi, yi,ji, cmap = colormap, norm=normalize,shading='auto') #let's look at it in logarithmic map
    cbar = pl.colorbar(change_j,ax=ax,fraction=0.03,extend='both') #both caps it at clim, but it can go further than that limit
    cbar.set_label(cbarlabel_deltajphi, rotation = 270,labelpad=30)
    cbar.mappable.set_clim(-100,100)
    
#     pl.title(''+str(m)+' : 1')
    pl.xlabel(xaxislabel)
    pl.ylabel(yaxislabel) #units: root Jr which will change shape into V

    pl.xlim(200,2500) #comment out for auto
    pl.ylim(0,150)    #originally 0,300
    
    pl.plot(Lz,-getJRfromJacobienergy(m)[0],color='green',ls='-',label='ILR')
    pl.plot(Lz,-getJRfromJacobienergy(m)[1],color='blue',ls='-',label='OLR')
    pl.plot(Lz,-getJRfromJacobienergy(m)[2],color='red',ls='-',label='CR')
    
    pl.legend(fontsize=10)
    pl.legend(bbox_to_anchor=(1.25,1.),loc='upper left')
    pl.show()
    
    
    
    ###################################################
    ##### Plot Jr vs Jphi color coded by delta Jr #####
    ###################################################
    
    
    fig, ax = pl.subplots()
    change_j = ax.pcolormesh(xi, yi,ji1, cmap = colormap, norm=normalize,shading='auto') #let's look at it in logarithmic map
    cbar = pl.colorbar(change_j,ax=ax,fraction=0.03,extend='both') #both caps it at clim, but it can go further than that limit
    cbar.set_label(cbarlabel_deltajr, rotation = 270,labelpad=30)
    cbar.mappable.set_clim(-50,50)
    
#     pl.title(''+str(m)+' : 1')
    pl.xlabel(xaxislabel)
    pl.ylabel(yaxislabel) #units: root Jr which will change shape into V

    pl.plot(Lz,-getJRfromJacobienergy(m)[0],color='green',ls='-',label='ILR')
    pl.plot(Lz,-getJRfromJacobienergy(m)[1],color='blue',ls='-',label='OLR')
    pl.plot(Lz,-getJRfromJacobienergy(m)[2],color='red',ls='-',label='CR')
    
    pl.legend(fontsize=10)
    pl.legend(bbox_to_anchor=(1.25,1.),loc='upper left')
    
    pl.xlim(200,2500) #comment out for auto
    pl.ylim(0,150)    #originally 0,300
    
    pl.show()
    # look at Jphi and jz
    
   
    
##################################################
#####  Command to make and print the graphs  #####
##################################################
    
        
#Total    
getangularactiongraph(nbins=101, colormap='coolwarm', normalize=mc.Normalize(), CRs=1, m=4, jphi=jphi_f, jphi_quiet=jphi_o, jr=jR_f, jr_quiet=jR_o,cbarlabel_deltajphi='$\Delta\ J_{\phi_{total}}$', cbarlabel_deltajr='$\Delta\ J_{r_{total}}$', xaxislabel='$J_{\phi_{o}}$ (kpc km/s)',yaxislabel='$J_{r_{o}}$ (kpc km/s)', vlinelabelILR='ILR', vlinelabelOLR='OLR')

#Growth  
getangularactiongraph(nbins=101, colormap='coolwarm', normalize=mc.Normalize(), CRs=1, m=4, jphi=jphi_fg, jphi_quiet=jphi_o, jr=jR_fg, jr_quiet=jR_o,cbarlabel_deltajphi='$\Delta\ J_{\phi_{growth}}$', cbarlabel_deltajr='$\Delta\ J_{r_{growth}}$', xaxislabel='$J_{\phi_{o}}$ (kpc km/s)',yaxislabel='$J_{r_{o}}$ (kpc km/s)', vlinelabelILR='ILR', vlinelabelOLR='OLR')

#Decay  
getangularactiongraph(nbins=101, colormap='coolwarm', normalize=mc.Normalize(), CRs=1, m=4, jphi=jphi_f, jphi_quiet=jphi_fg, jr=jR_f, jr_quiet=jR_fg,cbarlabel_deltajphi='$\Delta\ J_{\phi_{decay}}$', cbarlabel_deltajr='$\Delta\ J_{r_{decay}}$', xaxislabel='$J_{\phi_{peak}}$ (kpc km/s)',yaxislabel='$J_{r_{peak}}$ (kpc km/s)', vlinelabelILR='ILR', vlinelabelOLR='OLR')